In [1]:
# installing requirements, uncomment and run if needed!
#!pip -q install ./python

In [2]:
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name="/data/Reacher_Linux/Reacher.x86_64")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
INPUT_SIZE = 33         # state size
NUM_ACTIONS = 4         # the number of possible actions
SEED = 13               # a random seed for reproducibility

In [4]:
import torch
from ddpg_agent import Agent

#create the agent
agent = Agent(state_size=33, action_size=4, random_seed=SEED)
agent.actor_local.load_state_dict(torch.load('trained_actor.pth'))
agent.actor_local.eval()
print(agent.actor_local)
agent.critic_local.load_state_dict(torch.load('trained_critic.pth'))
agent.critic_local.eval()
print(agent.critic_local)

Actor(
  (fc0): Linear(in_features=33, out_features=128, bias=True)
  (bn0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)
Critic(
  (fcs1): Linear(in_features=33, out_features=256, bias=True)
  (fc2): Linear(in_features=260, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)


In [5]:
for i in range(10):
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment, train_mode true for speed
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        #just here, expand state to shape (1,33) for our bn
        action = agent.act(np.expand_dims(state,axis=0))
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    print("Episode: {} Score: {}".format(i, score))

Episode: 0 Score: 38.98999912850559
Episode: 1 Score: 35.029999217018485
Episode: 2 Score: 26.229999413713813
Episode: 3 Score: 34.259999234229326
Episode: 4 Score: 29.909999331459403
Episode: 5 Score: 31.52999929524958
Episode: 6 Score: 25.75999942421913
Episode: 7 Score: 35.5399992056191
Episode: 8 Score: 35.67999920248985
Episode: 9 Score: 34.20999923534691


In [6]:
env.close()